# Data Preparation & Target-Oriented EDA

Este notebook tem como objetivo preparar os dados para modelagem,
agora que o problema e o target foram formalmente definidos.

Neste estágio:
- o target é criado explicitamente
- decisões de tratamento são tomadas com base no objetivo preditivo
- colunas com risco de data leakage são removidas
- o dataset final ainda NÃO é convertido para formato de modelo

O output deste notebook é um dataframe limpo, consistente e conceitualmente válido.


In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

from src.features.engineering import add_month_cyclical_features
from src.features.target import build_delay_probability, binarize_target
from src.data.clean import drop_invalid_rows, drop_missing
from src.data.load import load_raw_data
from src.data.clean import drop_leakage_columns
from src import config
df = load_raw_data()

df = drop_invalid_rows(df)
df = build_delay_probability(df)
df = binarize_target(df, config.TARGET_QUANTILE)

df = drop_leakage_columns(df, config.LEAKAGE_COLS)
df = add_month_cyclical_features(df)
df = drop_missing(df)

In [4]:
df.shape

(317271, 14)

## Construção do target

O target do projeto é a **probabilidade de atraso superior a 15 minutos**,
definida como a razão entre o número de voos atrasados e o total de voos,
agregada por aeroporto, companhia aérea e período temporal.

Target operacional:
$$delay\_probability = \frac{arr\_del15}{arr\_flights}$$

Essa normalização evita viés de escala associado ao volume de operações.



In [5]:
df = df.copy()

# evita divisão por zero
df = df[df["arr_flights"] > 0]

df["delay_probability"] = df["arr_del15"] / df["arr_flights"]

df["delay_probability"].describe()

df["delay_probability"].isna().mean()

np.float64(0.0)

## Remoção de colunas pós-evento (data leakage)

As colunas abaixo representam decomposição ou explicação do atraso
após sua ocorrência e, portanto, não podem ser usadas como features
em um modelo preditivo legítimo.


In [4]:
df = df.drop(columns=config.LEAKAGE_COLS, errors="ignore")

df.columns

Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'arr_cancelled', 'arr_diverted',
       'delay_probability'],
      dtype='object')

## Decisão sobre valores ausentes

Após a criação do target e remoção de colunas com leakage,
o percentual de valores ausentes permanece baixo (<0.3%).

Opta-se por remover as linhas restantes com valores ausentes,
pois:
- o impacto no volume de dados é mínimo
- evita-se introduzir viés por imputação arbitrária


In [5]:
df= df.dropna()
df.shape

(317271, 11)

## Features mantidas para modelagem futura

As seguintes colunas são consideradas **conceitualmente válidas**:
- Identificadores temporais (`year`, `month`)
- Identificadores categóricos (`airport`, `carrier`)
- Volume operacional (`arr_flights`)


In [6]:
FEATURE_COLS = [
    "year",
    "month",
    "airport",
    "carrier",
    "arr_flights",
]

TARGET_COL = "delay_probability"

df[FEATURE_COLS + [TARGET_COL]].head()

,year,month,airport,carrier,arr_flights,delay_probability
0,2022,5,ABE,9E,136.0,0.051471
1,2022,5,ABY,9E,91.0,0.175824
2,2022,5,ACK,9E,19.0,0.105263
3,2022,5,AEX,9E,88.0,0.159091
4,2022,5,AGS,9E,181.0,0.104972


In [7]:
assert df[TARGET_COL].between(0, 1).all(), "Target fora do intervalo esperado"
assert df[FEATURE_COLS].isna().sum().sum(
) == 0, "Missing inesperado nas features"

## Output do Notebook 02

Este notebook produz um dataset:
- com target definido e validado
- sem colunas com data leakage
- com features conceitualmente válidas
- pronto para preparação final e modelagem

O próximo notebook será responsável por:
- encoding
- split temporal
- definição de X e y
